https://krpc.github.io/krpc/tutorials/suborbital-flight.html

In [5]:
import krpc
import time

In [2]:
conn = krpc.connect(name='Sub-orbital flight')

In [3]:
vessel = conn.space_center.active_vessel

In [11]:
vessel.resources.amount("SolidFuel")

30.0

In [7]:
# set pitch and heading to 90, keeping it upwards
vessel.auto_pilot.target_pitch_and_heading(90, 90)
vessel.auto_pilot.engage()
# set max throttle
vessel.control.throttle = 1
# sleep 1 sec to let the settings take effect
time.sleep(1)

print('Launch!')
vessel.control.activate_next_stage()

## Jettison booster by an event
# fuel_amount = conn.get_call(vessel.resources.amount, 'SolidFuel')
# expr = conn.krpc.Expression.less_than(
#     conn.krpc.Expression.call(fuel_amount),
#     conn.krpc.Expression.constant_float(0.1))
# event = conn.krpc.add_event(expr)
# with event.condition:
#     event.wait()
# print('Booster separation')
# vessel.control.activate_next_stage()


## Wait till altitude is higher than 10000, then do a gravity turn
mean_altitude = conn.get_call(getattr, vessel.flight(), 'mean_altitude')
expr = conn.krpc.Expression.greater_than(
    conn.krpc.Expression.call(mean_altitude),
    conn.krpc.Expression.constant_double(10000))
event = conn.krpc.add_event(expr)
with event.condition:
    event.wait()
    
print('Gravity turn')
vessel.auto_pilot.target_pitch_and_heading(60, 90)    

## Wait till apoapsis reach 100000, then reduce the throttle to 0.
apoapsis_altitude = conn.get_call(getattr, vessel.orbit, 'apoapsis_altitude')
expr = conn.krpc.Expression.greater_than(
    conn.krpc.Expression.call(apoapsis_altitude),
    conn.krpc.Expression.constant_double(100000))
event = conn.krpc.add_event(expr)
with event.condition:
    event.wait()

print('Launch stage separation')
# stop throttle
vessel.control.throttle = 0
time.sleep(1)
# next stage
vessel.control.activate_next_stage()
# turn off autopilot
vessel.auto_pilot.disengage()

## surface altitude, and activate next stage(landing)
# when surface altitude reach 1000
srf_altitude = conn.get_call(getattr, vessel.flight(), 'surface_altitude')
expr = conn.krpc.Expression.less_than(
    conn.krpc.Expression.call(srf_altitude),
    conn.krpc.Expression.constant_double(1000))
event = conn.krpc.add_event(expr)
with event.condition:
    event.wait()

# activate landing stage
vessel.control.activate_next_stage()
# when velocity reaches -0.1, declare landing.
while vessel.flight(vessel.orbit.body.reference_frame).vertical_speed < -0.1:
    print('Altitude = %.1f meters' % vessel.flight().surface_altitude)
    time.sleep(1)
print('Landed!')

Launch!
Gravity turn
Launch stage separation
Landed!
